In [61]:
import soccerdata as sd
from tqdm import tqdm

In [65]:
stats_types = ["standard", "shooting", "passing", "passing_types", "goal_shot_creation", "defense", "possession", "playing_time", "misc"]

In [66]:
old_seasons = ["05-06", "06-07", "07-08", "08-09","09-10", "10-11", "11-12", "12-13", "13-14", "14-15", "15-16", "16-17"]
new_seasons = ["17-18", "18-19", "19-20","20-21", "21-22", "22-23"]

old_fbref = sd.FBref(leagues="Big 5 European Leagues Combined", seasons=old_seasons)
new_fbref = sd.FBref(leagues="Big 5 European Leagues Combined", seasons=new_seasons)

[07/02/23 01:37:25] INFO     Saving cached data to /home/gabrielmmf/soccerdata/data/FBref             ]8;id=375062;file:///home/gabrielmmf/anaconda3/envs/env1/lib/python3.9/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=538927;file:///home/gabrielmmf/anaconda3/envs/env1/lib/python3.9/site-packages/soccerdata/_common.py#89\89]8;;\

                    INFO     Saving cached data to /home/gabrielmmf/soccerdata/data/FBref             ]8;id=125151;file:///home/gabrielmmf/anaconda3/envs/env1/lib/python3.9/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=201288;file:///home/gabrielmmf/anaconda3/envs/env1/lib/python3.9/site-packages/soccerdata/_common.py#89\89]8;;\

In [69]:
def reindex(df):
    df = df.reset_index()
    multiindex = df.columns
    index = ['_'.join(level) if level[1] != '' else level[0]
             for level in multiindex]

    df = df.set_axis(index, axis=1)
    year_mapping = {
        "0304": "2003",
        "0405": "2004",
        "0506": "2005",
        "0607": "2006",
        "0708": "2007",
        "0809": "2008",
        "0910": "2009",
        "1011": "2010",
        "1112": "2011",
        "1213": "2012",
        "1314": "2013",
        "1415": "2014",
        "1516": "2015",
        "1617": "2016",
        "1718": "2017",
        "1819": "2018",
        "1920": "2019",
        "2021": "2020",
        "2122": "2021",
        "2223": "2022"
    }

    next_year_mapping = {
        "2003": "2004",
        "2004": "2005",
        "2005": "2006",
        "2006": "2007",
        "2007": "2008",
        "2008": "2009",
        "2009": "2010",
        "2010": "2011",
        "2011": "2012",
        "2012": "2013",
        "2013": "2014",
        "2014": "2015",
        "2015": "2016",
        "2016": "2017",
        "2017": "2018",
        "2018": "2019",
        "2019": "2020",
        "2020": "2021",
        "2021": "2022",
        "2022": "2023"
    }
    df["season"] = df["season"].replace(year_mapping)
    df["nextSeason"] = df["season"].replace(next_year_mapping)

    df['season'] = df['season'].astype('int')
    df['nextSeason'] = df['nextSeason'].astype('int')
    return df


In [70]:
for type in tqdm(stats_types, total=len(stats_types)):
    reindex(new_fbref.read_player_season_stats(stat_type=type)).to_csv('data/new_fbref/' + type + '.csv', index=False)
    reindex(old_fbref.read_player_season_stats(stat_type=type)).to_csv('data/old_fbref/' + type + '.csv', index=False)

100%|██████████| 9/9 [05:57<00:00, 39.73s/it]
